In [1]:
"""
File v532_optima_base
Atasoy et al., 2013

Michel Bierlaire
Wed Aug  5 10:06:42 2020
"""

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta

# Read the data
df = pd.read_csv('optima.dat', '\t')
database = db.Database('optima', df)

# The following statement allows you to use the names of the
# variable as Python variable.
globals().update(database.variables)

# Removing some observations
exclude = Choice == -1
database.remove(exclude)

# Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_SM = Beta('ASC_SM', 0, None, None, 0)
BETA_COST = Beta('BETA_COST', 0, None, None, 0)
BETA_TIME_CAR = Beta('BETA_TIME_CAR', 0, None, None, 0)
BETA_TIME_PT = Beta('BETA_TIME_PT', 0, None, None, 0)
BETA_DIST = Beta('BETA_DIST', 0, None, None, 0)
BETA_NbCar = Beta('BETA_NbCar', 0, None, None, 0)
BETA_NbChild = Beta('BETA_NbChild', 0, None, None, 0)
BETA_LANGUAGE = Beta('BETA_LANGUAGE', 0, None, None, 0)
BETA_WorkTrip = Beta('BETA_WorkTrip', 0, None, None, 0)
BETA_Urban = Beta('BETA_Urban', 0, None, None, 0)
BETA_Student = Beta('BETA_Student', 0, None, None, 0)
BETA_Nbikes = Beta('BETA_Nbikes', 0, None, None, 0)


# Define new variables
FrenchRegion = LangCode == 1 
WORK = ((TripPurpose == 1) + (TripPurpose == 2)) > 0 #i.e TripPurpose = 1 or TripPurpose = 2
URBAN = UrbRur == 2 
STUDENT = OccupStat == 8 
NbCars = NbCar * (NbCar > 0) 
NbBikes = NbBicy * (NbBicy > 0) 
NbChildren = NbChild * (NbChild > 0) 

# Definition of the utility functions
CAR = ASC_CAR + \
    BETA_COST * CostCarCHF + \
    BETA_TIME_CAR * TimeCar + \
    BETA_NbCar * NbCars + \
    BETA_NbChild * NbChildren + \
    BETA_LANGUAGE * FrenchRegion + \
    BETA_WorkTrip * WORK

PT = BETA_COST * MarginalCostPT + \
    BETA_TIME_PT * TimePT + \
    BETA_Urban * URBAN + \
    BETA_Student * STUDENT

SM = ASC_SM + \
    BETA_DIST * distance_km + \
    BETA_Nbikes * NbBikes

# Associate utility functions with the numbering of alternatives
V = {0: PT, 1: CAR, 2: SM}

# The choice model is a logit, without availability conditions
logprob = models.loglogit(V, None, Choice)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'v532_optima_base'

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)


                  Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR       -0.413373  0.169202  -2.443073  1.456278e-02      0.173237   
ASC_SM        -0.470083  0.253323  -1.855669  6.350069e-02      0.369010   
BETA_COST     -0.059161  0.007482  -7.907476  2.664535e-15      0.010549   
BETA_DIST     -0.227327  0.020494 -11.092396  0.000000e+00      0.053077   
BETA_LANGUAGE  1.092534  0.162460   6.724934  1.756728e-11      0.158659   
BETA_NbCar     1.000978  0.089505  11.183542  0.000000e+00      0.097181   
BETA_NbChild   0.153526  0.066663   2.303000  2.127884e-02      0.064728   
BETA_Nbikes    0.346870  0.057122   6.072390  1.260203e-09      0.054750   
BETA_Student   3.206848  0.344729   9.302516  0.000000e+00      0.343505   
BETA_TIME_CAR -0.029929  0.003028  -9.883837  0.000000e+00      0.006035   
BETA_TIME_PT  -0.012068  0.001609  -7.499561  6.394885e-14      0.002653   
BETA_Urban     0.286151  0.125537   2.279410  2.264271e-02      0.122868   
BETA_WorkTri